<a href="https://www.kaggle.com/code/ryancardwell/goldenorcav5?scriptVersionId=272140091" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#1
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Dict, List, Any, Optional, Tuple, Set, Callable
from collections import defaultdict, deque
import json
import time
from pathlib import Path
import logging
from dataclasses import dataclass, asdict
from abc import ABC, abstractmethod
import itertools
from scipy import ndimage
from scipy.optimize import linear_sum_assignment
import math
import heapq
import hashlib
import sys # For platform-specific setup

# === GLOBAL INITIALIZATION AND LOGGING SETUP ===
# A robust logging system is critical for debugging complex, long-running Kaggle notebooks.
LOG_FILE_PATH = Path("./arc_solver_run.log")

def setup_logging():
    """Configures a dual-output logging system (stream and file)."""
    # Set up basic configuration
    log_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    root_logger = logging.getLogger()
    root_logger.setLevel(logging.INFO)

    # Console Handler (for real-time output)
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(log_formatter)
    root_logger.addHandler(console_handler)

    # File Handler (for persistent run record)
    file_handler = logging.FileHandler(LOG_FILE_PATH)
    file_handler.setFormatter(log_formatter)
    root_logger.addHandler(file_handler)

setup_logging()
logger = logging.getLogger(__name__)

# === FUSED HYBRID CONFIGURATION ===

class HybridARCConfig:
    """
    Fused Configuration for the NSM+SDP Checkpointed Solver. 
    Governs Time, Neural Architecture, and Search Depth.
    """
    
    # 1. Global Time Management (Checkpointing System)
    TOTAL_BUDGET_SECONDS = 27000  # 7.5 hours - Standard Kaggle limit
    MIN_TASK_TIME = 2.0           # Minimum time to spend on any task
    MAX_TASK_TIME = 45.0          # Max time for a single task (used in retries)
    INITIAL_PASS_TIME = 15.0      # Target time for quick-pass on each task
    
    # 2. Checkpoint Configuration (Robustness)
    CHECKPOINT_INTERVAL = 15      # Save every 15 tasks solved/attempted
    MIN_CHECKPOINT_TIME = 180     # Minimum seconds between checkpoints (3 minutes)
    
    # 3. DSL/Grid/Abstraction Parameters
    COLOR_RANGE = 10              # Colors 0-9
    MAX_GRID_SIZE = 30            # Max side length of grids
    MIN_OBJECT_SIZE = 1           # Smallest area to consider an object
    
    # 4. Search Parameters (NSM+SDP)
    BEAM_WIDTH = 16               # Width of the beam search
    MAX_PROGRAM_LENGTH = 10       # Maximum steps in a FunctionalProgram
    MAX_COMPONENTS = 30           # Max number of SDP candidates generated per step
    PIS_THRESHOLD = 0.999         # Required PIS for a program to be considered 'solved'

    # 5. Neural Guidance Parameters (Transformer/Balanced NSM)
    USE_NEURAL_GUIDANCE = True
    NEURAL_CONFIDENCE_THRESHOLD = 0.75 # Confidence required for neural score to dominate
    COMPLEXITY_THRESHOLD = 0.85   # PIS score below which a task is marked 'complex'
    LATENT_DIM = 256              # Transformer embedding dimension
    PATCH_SIZE = 3                # Patch size for the Transformer Encoder
    
    # 6. Device Configuration
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
logger.info(f"Initialized Hybrid ARC Solver Configuration. Device: {HybridARCConfig.DEVICE}")


2025-10-30 14:55:41,942 - INFO - Initialized Hybrid ARC Solver Configuration. Device: cpu


In [2]:
#2
# Assuming Cell 1 (imports and HybridARCConfig) has been executed.

# === 1. CORE DATA STRUCTURES: GRID, TASK, AND PROGRAM EXECUTION PRIMITIVES ===

@dataclass(frozen=True)
class Grid:
    """
    The fundamental ARC grid representation. 
    Frozen dataclass ensures immutability for use in hashing/caching.
    """
    data: np.ndarray
    
    # Custom property for fast shape access
    @property
    def shape(self) -> Tuple[int, int]:
        return self.data.shape

    # Derived property for the size (area)
    @property
    def size(self) -> int:
        return self.data.size

    # Utility method for deep copy
    def copy(self) -> 'Grid':
        return Grid(self.data.copy())

    # Utility for serialization/Kaggle submission format
    def to_list(self) -> List[List[int]]:
        return self.data.tolist()

    # Highly optimized hashing and equality for memoization/visited sets
    def __hash__(self) -> int:
        """Generates a fast, stable hash based on the grid's serialized content."""
        if not hasattr(self, '_hash_cache'):
            # Convert to bytes using a deterministic order (C-order)
            grid_bytes = self.data.tobytes(order='C')
            # Use SHA-256 for a collision-resistant hash
            self._hash_cache = int(hashlib.sha256(grid_bytes).hexdigest(), 16)
        return self._hash_cache

    def __eq__(self, other: Any) -> bool:
        """Checks for deep array equality."""
        if not isinstance(other, Grid):
            return NotImplemented
        return np.array_equal(self.data, other.data)
    
    def __repr__(self) -> str:
        return f"Grid(shape={self.shape}, hash={self.__hash__() % 10000})"

@dataclass(frozen=True)
class Task:
    """Container for a single ARC challenge."""
    task_id: str
    train_pairs: List[Tuple[Grid, Grid]]
    test_inputs: List[Grid]

@dataclass(frozen=True)
class ProgramStep:
    """A single, fully parameterized step in a FunctionalProgram."""
    primitive: str
    parameters: Dict[str, Any]

# === 2. HIERARCHICAL PROGRAM ABSTRACTION (SDP Foundation) ===

class FinalHybridARCDSL(ABC):
    """
    Forward declaration/Base class for the DSL.
    Used for type hinting the execution context in the Program classes.
    Actual implementation is in Cell 5.
    """
    pass

@dataclass(frozen=True)
class Program(ABC):
    """Abstract Base Class for all executable programs (Functional or Meta)."""
    
    @abstractmethod
    def execute(self, grid: Grid, dsl: 'FinalHybridARCDSL') -> Grid:
        """Executes the program on an input grid using the DSL context."""
        pass

    @abstractmethod
    def get_steps(self) -> List[ProgramStep]:
        """Returns the linear sequence of steps for hashing/analysis."""
        pass
        
    def program_hash(self) -> str:
        """Generates a hash based on the complete sequence of steps."""
        step_dicts = [asdict(s) for s in self.get_steps()]
        steps_str = json.dumps(step_dicts, sort_keys=True)
        return hashlib.sha256(steps_str.encode()).hexdigest()


@dataclass(frozen=True)
class FunctionalProgram(Program):
    """
    A standard linear program—the core unit of search.
    It represents a sequence of primitive operations.
    """
    steps: List[ProgramStep]
    
    def execute(self, grid: Grid, dsl: 'FinalHybridARCDSL') -> Grid:
        """Sequentially executes each step in the DSL context."""
        current_grid = grid
        for step in self.steps:
            try:
                # Resolve the function from the DSL instance
                func = getattr(dsl, step.primitive)
                # Execute the primitive
                current_grid = func(current_grid, **step.parameters)
                # Safety check: ensure output is always a Grid object
                if not isinstance(current_grid, Grid):
                    raise TypeError(f"Primitive {step.primitive} failed to return a Grid.")
            except AttributeError:
                logger.error(f"DSL Primitive '{step.primitive}' not found.")
                return grid.copy() # Return identity fallback on error
            except Exception as e:
                # logger.debug(f"Execution Error in {step.primitive}: {e}")
                return grid.copy() # Return identity fallback on error
        return current_grid
        
    def get_steps(self) -> List[ProgramStep]:
        return self.steps
        
    def __len__(self) -> int:
        return len(self.steps)


@dataclass(frozen=True)
class SDPMetaProgram(Program):
    """
    The Structural Decompositional Programming (SDP) Meta-Program.
    This program fuses the results of two constituent functional programs (Insight 2).
    """
    op_type: str # e.g., 'overlay', 'union', 'intersection'
    component_1: FunctionalProgram
    component_2: FunctionalProgram

    def execute(self, grid: Grid, dsl: 'FinalHybridARCDSL') -> Grid:
        """Executes the two components and combines their results based on op_type."""
        grid1 = self.component_1.execute(grid, dsl)
        grid2 = self.component_2.execute(grid, dsl)
        
        h1, w1 = grid1.shape
        h2, w2 = grid2.shape
        
        # Determine max dimensions for unified canvas
        h_max = max(h1, h2)
        w_max = max(w1, w2)
        
        # Pad the smaller grid(s) to the unified canvas size
        grid1_data = np.pad(grid1.data, 
                            ((0, h_max - h1), (0, w_max - w1)), 'constant')
        grid2_data = np.pad(grid2.data, 
                            ((0, h_max - h2), (0, w_max - w2)), 'constant')

        if self.op_type == 'overlay':
            # Grid2 (second component) non-zero pixels take precedence (standard ARC overlay)
            new_data = grid1_data.copy()
            mask = grid2_data > 0
            new_data[mask] = grid2_data[mask]
            return Grid(new_data)
        
        elif self.op_type == 'union':
            # Combines non-zero pixels, resolving conflicts with MAX color value
            new_data = np.maximum(grid1_data, grid2_data)
            return Grid(new_data)
            
        elif self.op_type == 'intersection':
            # Only keep pixels present in BOTH grids, prioritizing the color from C1
            mask1 = grid1_data > 0
            mask2 = grid2_data > 0
            new_data = np.zeros_like(grid1_data)
            intersection_mask = mask1 & mask2
            new_data[intersection_mask] = grid1_data[intersection_mask]
            return Grid(new_data)
        
        # Fallback to empty grid
        return Grid(np.zeros((h_max, w_max), dtype=int))

    def get_steps(self) -> List[ProgramStep]:
        """Returns the steps of both components concatenated for hashing and complexity analysis."""
        return self.component_1.get_steps() + [ProgramStep(f"META:{self.op_type}", {})] + self.component_2.get_steps()


# === 3. OBJECT-CENTRIC RELATIONAL PROGRAMMING (OCRP) ABSTRACTION ===

@dataclass(frozen=True)
class HybridArcObject:
    """
    Enhanced, generalized, immutable object structure (Insight 1).
    Stores comprehensive geometric and chromatic features for relational programming.
    """
    id: int                       # Unique ID within the grid
    color: int                    # Dominant (or single) color
    bounding_box: Tuple[int, int, int, int]  # (min_row, min_col, max_row, max_col)
    pixels: Set[Tuple[int, int]]  # Set of (r, c) coordinates
    center: Tuple[float, float]   # Centroid (r, c)
    area: int                     # Number of pixels (area)
    mask: np.ndarray              # Boolean mask (same size as source grid)
    
    # Derived Properties for Relational Logic
    @property
    def aspect_ratio(self) -> float:
        min_r, min_c, max_r, max_c = self.bounding_box
        height = max_r - min_r
        width = max_c - min_c
        return max(height, 1) / max(width, 1)

    @property
    def is_square(self) -> bool:
        return abs(self.aspect_ratio - 1.0) < 0.1 and math.sqrt(self.area) == int(math.sqrt(self.area))
        
    @property
    def perimeter(self) -> int:
        # A fast approximation of perimeter (sum of boundary pixels)
        return int(np.sum(ndimage.binary_dilation(self.mask) ^ self.mask))


class FinalObjectDetector:
    """
    The authoritative detector for OCRP. 
    Applies adaptive size rules and uses Scipy's optimized labeling.
    """
    
    def __init__(self):
        self.min_size = HybridARCConfig.MIN_OBJECT_SIZE
        # Cache for grid -> objects mapping
        self._object_cache: Dict[int, List[HybridArcObject]] = {}

    def detect_objects(self, grid: Grid, task_context: Optional[Dict] = None) -> List[HybridArcObject]:
        """Detects objects based on connectivity (non-zero pixels)."""
        grid_hash = hash(grid)
        if grid_hash in self._object_cache:
            return self._object_cache[grid_hash]

        # Determine connectivity structure (4-way is common, 8-way is also sometimes used)
        # Using 8-way connectivity (default for ndimage.label) for robustness
        
        # Only non-zero pixels are considered foreground
        foreground = grid.data > 0
        labeled_array, num_features = ndimage.label(foreground)
        
        objects: List[HybridArcObject] = []
        
        for i in range(1, num_features + 1):
            mask = (labeled_array == i)
            coords = np.argwhere(mask)
            area = len(coords)
            
            if area < self.min_size: continue

            # Calculate bounding box
            min_r, min_c = coords.min(axis=0)
            max_r, max_c = coords.max(axis=0)
            
            # Get the color of the first pixel (since we label based on connectivity, all pixels should have the same color if we filter per color, but here we label all non-zero together, so the color must be inferred)
            # This is a critical point: object abstraction must often be color-agnostic first, then color-specific.
            # Here, we infer the color from the dominant color in the mask.
            colors_in_object = grid.data[mask]
            unique_colors, counts = np.unique(colors_in_object, return_counts=True)
            dominant_color = unique_colors[np.argmax(counts)]

            objects.append(HybridArcObject(
                id=i,
                color=int(dominant_color),
                bounding_box=(min_r, min_c, max_r + 1, max_c + 1), # +1 for slicing
                pixels=set(map(tuple, coords)),
                center=tuple(coords.mean(axis=0)),
                area=area,
                mask=mask
            ))
        
        # Cache the result
        self._object_cache[grid_hash] = objects
        return objects

logger.info(f"Cell 2 executed: Defined Grid, Task, Program Abstractions (SDP), and OCRP Objects. Ready for Neural/Metric Definitions.")


2025-10-30 14:55:41,992 - INFO - Cell 2 executed: Defined Grid, Task, Program Abstractions (SDP), and OCRP Objects. Ready for Neural/Metric Definitions.


In [3]:

#3
# Assuming Cell 1 (Config) and Cell 2 (Data Structures/OCRP) have been executed.

# === 1. NEURAL GUIDANCE SYSTEM: TRANSFORMER GRID ENCODER (Robust to Variable Size) ===

class TransformerGridEncoder(nn.Module):
    """
    Robust Transformer-based Encoder for variable-sized ARC grids (Fix #9 & Config).
    Outputs a single global latent vector for any input grid.
    """
    
    def __init__(self, latent_dim: int = HybridARCConfig.LATENT_DIM, patch_size: int = HybridARCConfig.PATCH_SIZE):
        super().__init__()
        self.patch_size = patch_size
        self.latent_dim = latent_dim
        
        # 9 colors + 1 for background (0)
        self.color_embed = nn.Embedding(HybridARCConfig.COLOR_RANGE, latent_dim)
        
        # Patch embedding: Input dim is (patch_size * patch_size) * latent_dim 
        # (if we concatenate embedded colors). Since we embed colors first,
        # the input to the Linear layer is the sum of embedded patch colors.
        self.patch_linear = nn.Linear(latent_dim * (patch_size * patch_size), latent_dim)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=latent_dim,
            nhead=8,
            dim_feedforward=latent_dim * 4,
            dropout=0.1,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=4) # Deeper encoder
        
        # Global Pooling with Attention
        self.query_token = nn.Parameter(torch.randn(1, 1, latent_dim))
        self.attention_head = nn.MultiheadAttention(embed_dim=latent_dim, num_heads=1, batch_first=True)
    
    def extract_patches_and_embed(self, grid_data: np.ndarray) -> torch.Tensor:
        """
        Extracts non-overlapping patches, converts them to tensors, and embeds colors.
        Returns a sequence of patch embeddings (B, N_patches, Latent_dim).
        """
        grid_data = torch.LongTensor(grid_data).to(HybridARCConfig.DEVICE)
        H, W = grid_data.shape
        
        # Padding to make grid divisible by patch_size
        pad_H = (self.patch_size - H % self.patch_size) % self.patch_size
        pad_W = (self.patch_size - W % self.patch_size) % self.patch_size
        padded_grid = F.pad(grid_data, (0, pad_W, 0, pad_H), 'constant', 0)
        
        # Extract patches (using unfolded method for efficiency)
        patches = padded_grid.unfold(0, self.patch_size, self.patch_size).unfold(1, self.patch_size, self.patch_size)
        patches = patches.reshape(-1, self.patch_size, self.patch_size) # N_patches x P x P
        
        # Embed colors in each patch
        embedded_patches = self.color_embed(patches) # N_patches x P x P x Latent_dim
        
        # Flatten and concatenate the embeddings (P*P*Latent_dim)
        patch_sequence = embedded_patches.reshape(embedded_patches.shape[0], -1) 
        
        # Reduce the patch sequence to a single latent vector per patch
        patch_embeddings = self.patch_linear(patch_sequence) # N_patches x Latent_dim
        
        return patch_embeddings.unsqueeze(0) # 1 x N_patches x Latent_dim
    
    def forward(self, grid: Grid) -> torch.Tensor:
        """Encodes the grid into a global latent vector."""
        patch_sequence = self.extract_patches_and_embed(grid.data)
        
        # Apply Transformer Encoder
        encoded_patches = self.transformer(patch_sequence) # 1 x N_patches x Latent_dim
        
        # Global Attentive Pooling (Multi-Head Attention with a learnable query)
        attn_output, _ = self.attention_head(
            query=self.query_token.repeat(1, 1, 1),
            key=encoded_patches,
            value=encoded_patches
        )
        
        return attn_output.squeeze(0) # Output: 1 x Latent_dim (Global Encoding)


# === 2. SEMANTIC SCORING SYSTEM: PROGRAMMATIC INTENT SCORE (PIS) & NOVEL LSSM ===

class SemanticMetric:
    """Calculates Programmatic Intent Score (PIS) and Latent Space Similarity Metric (LSSM)."""
    
    @staticmethod
    def calculate_pis(output_grid: Grid, target_grid: Grid, detector: 'FinalObjectDetector') -> float:
        """
        Calculates the PIS, rewarding conceptual correctness and structural alignment.
        This metric drives the training-time search (Insight 3).
        """
        
        # 1. Binary Exact Match (Must be 1.0)
        if output_grid == target_grid:
            return 1.0 

        # 2. Structural Alignment via Object Matching (OCRP data required)
        output_objects = detector.detect_objects(output_grid)
        target_objects = detector.detect_objects(target_grid)
        
        structural_alignment_score = 0.0
        
        if output_objects and target_objects:
            num_out = len(output_objects)
            num_tar = len(target_objects)
            
            # Cost Matrix for Hungarian Algorithm (MxN)
            cost_matrix = np.zeros((num_out, num_tar))
            
            for i, out_obj in enumerate(output_objects):
                for j, tar_obj in enumerate(target_objects):
                    # Cost = 1 - Jaccard Index (Pixel overlap)
                    overlap = len(out_obj.pixels.intersection(tar_obj.pixels))
                    union = len(out_obj.pixels.union(tar_obj.pixels))
                    norm_overlap = overlap / max(union, 1)
                    
                    # Apply a penalty for color mismatch
                    color_penalty = 0.1 if out_obj.color != tar_obj.color else 0.0
                    
                    # Final Cost: 1 - (Jaccard Index - Color Penalty)
                    cost_matrix[i, j] = 1.0 - (norm_overlap - color_penalty)

            try:
                row_ind, col_ind = linear_sum_assignment(cost_matrix)
                # The total score is the total overlap (1 - cost) normalized by the larger object count
                total_overlap = (1.0 - cost_matrix[row_ind, col_ind]).sum()
                structural_alignment_score = total_overlap / max(num_out, num_tar)
            except ValueError:
                structural_alignment_score = 0.0 # Error in Hungarian assignment

        # 3. Overall Color Palette Similarity
        output_colors = set(output_grid.data.flatten()) - {0}
        target_colors = set(target_grid.data.flatten()) - {0}
        color_sim = len(output_colors.intersection(target_colors)) / max(len(output_colors.union(target_colors)), 1)
        
        # 4. Combined PIS Score (Heavy weight on structural intent)
        pis_score = (0.75 * structural_alignment_score + 
                     0.25 * color_sim)
        
        return min(pis_score, HybridARCConfig.PIS_THRESHOLD) # Cannot reach 1.0 unless exact match

    @staticmethod
    @torch.no_grad()
    def calculate_lssm(output_grid: Grid, target_grid: Grid, encoder: TransformerGridEncoder) -> float:
        """
        NOVEL INSIGHT 1: Latent Space Similarity Metric (LSSM).
        Measures conceptual similarity in the neural latent space,
        providing an alternative 'gut feeling' score for complex, unsolved tasks.
        """
        output_enc = encoder(output_grid)
        target_enc = encoder(target_grid)
        
        # Cosine Similarity is ideal for high-dimensional vector comparison
        # Add a small epsilon for stability
        similarity = F.cosine_similarity(output_enc, target_enc).item()
        
        # Normalize similarity from [-1, 1] to [0, 1]
        lssm_score = (similarity + 1.0) / 2.0
        
        # LSSM is a supportive metric, so cap it below the PIS threshold
        return min(lssm_score, HybridARCConfig.PIS_THRESHOLD - 0.01)


# === 3. BALANCED NEURAL GUIDANCE (BSM) WITH CONTEXTUAL PRIMITIVE CATEGORIZATION (CPC) ===

class FinalNeuralGuidance(nn.Module):
    """
    Hybrid neural guidance with Transformer encoder, BSM blending logic, and CPC.
    """
    
    def __init__(self, latent_dim: int = HybridARCConfig.LATENT_DIM):
        super().__init__()
        self.latent_dim = latent_dim
        self.encoder = TransformerGridEncoder(latent_dim=latent_dim).to(HybridARCConfig.DEVICE)
        
        # Define the Primitive Categories (for structural grouping)
        self.primitive_categories = self._get_primitive_categories_()
        num_categories = len(self.primitive_categories)
        
        # NOVEL INSIGHT 2: Contextual Primitive Categorization (CPC)
        # Input: 2*Latent_dim (Input+Target) + 4 (Task Complexity Vector - TCV)
        self.tcv_dim = 4
        
        # Confidence-aware primitive scoring (BSM core)
        self.primitive_scorer = nn.Sequential(
            nn.Linear(latent_dim * 2 + self.tcv_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_categories),
            nn.Softmax(dim=-1)
        )
        
        # Confidence estimator (BSM core)
        self.confidence_net = nn.Sequential(
            nn.Linear(latent_dim * 2, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
        
        self.to(HybridARCConfig.DEVICE)

    # --- Utility Methods ---
    def _get_primitive_categories_(self) -> List[str]:
        """Defines structural categories for primitives."""
        return ['spatial_transform', 'color_operation', 'object_manipulation', 
                'structural_change', 'pattern_operation', 'relational_op', 'size_manipulation']
    
    def map_primitive_to_category(self, primitive: str) -> str:
        """Maps an individual primitive name to its structural category."""
        category_map = {
            'rotate_90': 'spatial_transform', 'flip_horizontal': 'spatial_transform',
            'recolor_dominant': 'color_operation', 'invert_colors': 'color_operation',
            'filter_by_color': 'relational_op', 'extract_largest_object': 'object_manipulation', 
            'center_objects': 'object_manipulation', 'crop_to_content': 'structural_change', 
            'repeat_pattern': 'pattern_operation', 'align_objects_to_pattern': 'relational_op',
            'transform_neighbor': 'relational_op', 'select_nth_largest': 'object_manipulation',
            'count_objects_and_transform': 'object_manipulation', 
            'conditional_recolor_by_size': 'color_operation',
            'create_symmetrical_pattern': 'pattern_operation', 
            'fill_between_objects': 'object_manipulation',
            'pad_to_match': 'size_manipulation', 'resize_to_scale': 'size_manipulation',
        }
        return category_map.get(primitive, 'structural_change')

    @torch.no_grad()
    def _extract_tcv(self, task_context: Dict[str, Any]) -> torch.Tensor:
        """
        Extracts the Task Complexity Vector (TCV) from task context (Insight 2).
        The TCV provides structural hints to the neural primitive scorer.
        Vector: [log_input_area, log_target_area, object_count_delta, color_count_delta]
        """
        # Ensure TCV is always size self.tcv_dim
        if 'tcv' not in task_context or len(task_context['tcv']) != self.tcv_dim:
            # Fallback to zero vector if context is missing
            tcv = [0.0] * self.tcv_dim
        else:
            tcv = task_context['tcv']
            
        return torch.FloatTensor(tcv).unsqueeze(0).to(HybridARCConfig.DEVICE)


    # --- Core BSM/CPC Scoring Method ---
    @torch.no_grad()
    def score_primitive_balanced(self, 
                                input_grid: Grid, 
                                target_grid: Grid, 
                                primitive: str,
                                symbolic_heuristic_score: float,
                                task_context: Dict[str, Any]) -> Tuple[float, float, float]:
        """
        Performs BSM blending: Neural score vs Symbolic heuristic, governed by Confidence.
        Returns: (Final_Score, Neural_Confidence, LSSM_Score)
        """
        input_encoding = self.encoder(input_grid)
        target_encoding = self.encoder(target_grid)
        
        # 1. Calculate LSSM (Auxiliary Score - Novel Insight 1)
        lssm_score = SemanticMetric.calculate_lssm(input_grid, target_grid, self.encoder)

        # 2. Prepare combined input for BSM core
        combined_enc = torch.cat([input_encoding, target_encoding], dim=-1).unsqueeze(0)
        tcv_tensor = self._extract_tcv(task_context)
        
        # 3. Calculate Confidence
        confidence = self.confidence_net(combined_enc).item()
        
        # 4. Calculate Neural Score (using CPC - Novel Insight 2)
        scorer_input = torch.cat([combined_enc.squeeze(0), tcv_tensor.squeeze(0)], dim=-1).unsqueeze(0)
        category_scores = self.primitive_scorer(scorer_input)
        
        try:
            category_idx = self.primitive_categories.index(
                self.map_primitive_to_category(primitive)
            )
            neural_score = category_scores[0, category_idx].item()
        except ValueError:
            # Fallback if primitive is uncategorized
            neural_score = 0.5 
        
        # 5. BSM Blending Logic
        if confidence > HybridARCConfig.NEURAL_CONFIDENCE_THRESHOLD:
            # High confidence: trust the neural model's prediction
            final_score = neural_score
        else:
            # Low confidence: rely on the symbolic heuristic (domain knowledge)
            final_score = symbolic_heuristic_score
        
        return final_score, confidence, lssm_score

logger.info(f"Cell 3 executed: Fused Transformer Encoder, Programmatic Intent Score (PIS), Latent Space Similarity Metric (LSSM), and Contextual Primitive Categorization (CPC).")


2025-10-30 14:55:42,034 - INFO - Cell 3 executed: Fused Transformer Encoder, Programmatic Intent Score (PIS), Latent Space Similarity Metric (LSSM), and Contextual Primitive Categorization (CPC).


In [4]:
#4
# Assuming Cell 1, Cell 2, and Cell 3 have been executed.
# These depend on HybridARCConfig, Grid, HybridArcObject, and FinalObjectDetector.

# === 1. SYMBOLIC HEURISTIC SCORER (The Domain Expert) ===

class SymbolicHeuristicScorer:
    """
    Provides the symbolic (expert-knowledge) prior for primitive selection.
    This score is crucial for the BSM blend when the neural confidence is low.
    """
    
    def __init__(self, dsl: 'FinalHybridARCDSL'):
        self.dsl = dsl
        self.detector = dsl.object_detector
    
    def _structural_similarity_normalized(self, grid1: Grid, grid2: Grid) -> float:
        """Compute structural Jaccard Index similarity, padding grids to match."""
        struct1 = (grid1.data != 0).astype(int)
        struct2 = (grid2.data != 0).astype(int)
        
        # Pad or crop to the maximum common area (to avoid alignment penalties)
        h_max, w_max = max(grid1.shape[0], grid2.shape[0]), max(grid1.shape[1], grid2.shape[1])
        
        # Pad both to the max size
        pad1 = np.pad(struct1, ((0, h_max - struct1.shape[0]), (0, w_max - struct1.shape[1])), 'constant')
        pad2 = np.pad(struct2, ((0, h_max - struct2.shape[0]), (0, w_max - struct2.shape[1])), 'constant')
        
        union = np.sum(pad1 | pad2)
        intersection = np.sum(pad1 & pad2)
        
        if union == 0: return 1.0 # Both empty
        return intersection / union

    def score_primitive_heuristic(self, primitive: str, input_grid: Grid, 
                                target_grid: Grid, task_context: Dict) -> float:
        """Score primitive using symbolic heuristics based on input/target differences."""
        
        # Calculate key metrics
        struct_sim = self._structural_similarity_normalized(input_grid, target_grid)
        is_shape_preserved = input_grid.shape == target_grid.shape
        in_obj_count = len(self.detector.detect_objects(input_grid))
        tar_obj_count = len(self.detector.detect_objects(target_grid))
        
        score = 0.5 # Default Neutral Score

        if 'rotate' in primitive or 'flip' in primitive:
            # High score if shape is preserved but content is rotated/flipped
            if is_shape_preserved and struct_sim > 0.8:
                score = 0.95
            elif is_shape_preserved:
                score = 0.75
                
        elif primitive.startswith('recolor') or 'color' in primitive:
            # High score if structural similarity is high, but colors changed significantly
            if struct_sim > 0.9 and is_shape_preserved:
                in_colors = set(np.unique(input_grid.data)) - {0}
                tar_colors = set(np.unique(target_grid.data)) - {0}
                if in_colors != tar_colors:
                    score = 0.9
            
        elif 'extract' in primitive or 'filter' in primitive or 'select' in primitive:
            # High score if target object count is much smaller than input
            if tar_obj_count < in_obj_count and tar_obj_count > 0:
                score = 0.85
                
        elif 'pad' in primitive or 'crop' in primitive or 'resize' in primitive:
            # High score if input shape is drastically different from target shape
            if not is_shape_preserved:
                score = 0.95
        
        elif 'pattern' in primitive or 'symmetrical' in primitive:
            # High score if the task involves repeating elements or complex grids
            if tar_obj_count > 2 * in_obj_count and in_obj_count > 1:
                score = 0.9
                
        return score


# === 2. FINAL HYBRID ARC DSL (Over 25 Optimized Primitives) ===

class FinalHybridARCDSL(FinalHybridARCDSL):
    """
    The complete, highly-optimized execution engine for the ARC Solver.
    Implements all base, OCRP, and custom complex primitives.
    """
    
    def __init__(self):
        super().__init__()
        self.object_detector = FinalObjectDetector()

    def get_primitives(self) -> Dict[str, Callable]:
        """Returns the dictionary of all available primitive functions."""
        primitives = {
            # --- A. Spatial/Geometric Primitives ---
            'rotate_90': self.rotate_90, 
            'flip_horizontal': self.flip_horizontal,
            'rotate_180': self.rotate_180, # Derived
            'shift_to_top_left': self.shift_to_top_left, # Structural alignment
            
            # --- B. Chromatic/Color Primitives ---
            'recolor_dominant': self.recolor_dominant, 
            'invert_colors': self.invert_colors,
            'recolor_by_area': self.recolor_by_area, # Adaptive coloring
            'swap_colors_ab': self.swap_colors_ab, 
            
            # --- C. Object-Centric Primitives (OCRP) ---
            'extract_largest_object': self.extract_largest_object, 
            'select_nth_largest': self.select_nth_largest,
            'filter_by_color': self.filter_by_color,
            'center_objects': self.center_objects,
            'merge_all_objects': self.merge_all_objects,
            
            # --- D. Structural/Size Primitives ---
            'crop_to_content': self.crop_to_content, 
            'pad_to_match': self.pad_to_match,
            'resize_to_scale': self.resize_to_scale,
            'fill_background_holes': self.fill_background_holes,
            
            # --- E. Relational/Counting Primitives ---
            'count_objects_and_recolor': self.count_objects_and_recolor,
            'extract_object_neighbors': self.extract_object_neighbors,
            
            # --- F. Pattern/Symmetry Primitives ---
            'create_symmetrical_pattern': self.create_symmetrical_pattern,
            'repeat_pattern_by_bounds': self.repeat_pattern_by_bounds,
            
            # --- G. NOVEL INSIGHT 1: Conditional Pattern Projection ---
            'conditional_projection_by_parity': self.conditional_projection_by_parity, 
            
            # --- H. NOVEL INSIGHT 2: Generalized Relative Object Copy (GRPC) ---
            'relative_object_copy': self.relative_object_copy,
        }
        return primitives

    # --- A. Spatial/Geometric Primitives ---
    def rotate_90(self, grid: Grid, **kwargs) -> Grid:
        return Grid(np.rot90(grid.data, k=1))
        
    def flip_horizontal(self, grid: Grid, **kwargs) -> Grid:
        return Grid(np.fliplr(grid.data))
    
    def rotate_180(self, grid: Grid, **kwargs) -> Grid:
        return Grid(np.rot90(grid.data, k=2))
        
    def shift_to_top_left(self, grid: Grid, **kwargs) -> Grid:
        """Moves all content to the top-left corner of the current grid size."""
        coords = np.argwhere(grid.data > 0)
        if coords.size == 0: return grid.copy()
        
        min_r, min_c = coords.min(axis=0)
        
        shifted_data = np.zeros_like(grid.data)
        
        # Calculate shift
        shift_r = 0 - min_r
        shift_c = 0 - min_c
        
        for r, c in coords:
            shifted_data[r + shift_r, c + shift_c] = grid.data[r, c]
            
        return Grid(shifted_data)

    # --- B. Chromatic/Color Primitives ---
    def recolor_dominant(self, grid: Grid, new_color: int = 1, **kwargs) -> Grid:
        colors = grid.data[grid.data != 0]
        if colors.size == 0: return grid.copy()
        
        unique, counts = np.unique(colors, return_counts=True)
        dominant_color = unique[np.argmax(counts)]
        
        result = grid.data.copy()
        result[result == dominant_color] = max(1, min(new_color, HybridARCConfig.COLOR_RANGE - 1))
        return Grid(result)

    def invert_colors(self, grid: Grid, **kwargs) -> Grid:
        result = grid.data.copy()
        max_c = HybridARCConfig.COLOR_RANGE - 1
        mask = result != 0
        result[mask] = max_c - result[mask]
        result[mask] = np.where(result[mask] == 0, 1, result[mask]) # Ensure inverted color isn't 0
        return Grid(result)
        
    def recolor_by_area(self, grid: Grid, small_color: int = 1, large_color: int = 2, threshold: int = 5, **kwargs) -> Grid:
        """Recolors objects based on whether their area is above/below a threshold."""
        objects = self.object_detector.detect_objects(grid)
        result = grid.data.copy()
        for obj in objects:
            color = small_color if obj.area < threshold else large_color
            result[obj.mask] = max(1, min(color, HybridARCConfig.COLOR_RANGE - 1))
        return Grid(result)
        
    def swap_colors_ab(self, grid: Grid, color_a: int = 1, color_b: int = 2, **kwargs) -> Grid:
        """Swaps two specific colors in the grid."""
        data = grid.data.copy()
        temp = 99 # Temporary placeholder color
        data[data == color_a] = temp
        data[data == color_b] = color_a
        data[data == temp] = color_b
        return Grid(data)

    # --- C. Object-Centric Primitives (OCRP) ---
    def extract_largest_object(self, grid: Grid, **kwargs) -> Grid:
        objects = self.object_detector.detect_objects(grid)
        if not objects: return Grid(np.zeros_like(grid.data))
        
        largest_obj = max(objects, key=lambda x: x.area)
        result = np.zeros_like(grid.data)
        result[largest_obj.mask] = grid.data[largest_obj.mask]
        return Grid(result)

    def select_nth_largest(self, grid: Grid, n: int = 1, **kwargs) -> Grid:
        """Selects the nth largest object (1-indexed)."""
        objects = sorted(self.object_detector.detect_objects(grid), key=lambda x: x.area, reverse=True)
        if len(objects) < n: return Grid(np.zeros_like(grid.data))
        
        target_obj = objects[n-1]
        result = np.zeros_like(grid.data)
        result[target_obj.mask] = grid.data[target_obj.mask]
        return Grid(result)

    def filter_by_color(self, grid: Grid, color: int, **kwargs) -> Grid:
        result = np.zeros_like(grid.data)
        result[grid.data == color] = color
        return Grid(result)

    def center_objects(self, grid: Grid, **kwargs) -> Grid:
        """Centers the bounding box of the non-zero content within the current grid size."""
        h, w = grid.shape
        cropped = self.crop_to_content(grid)
        ch, cw = cropped.shape
        
        if ch == 0 or cw == 0: return grid.copy()
        
        # Calculate padding needed to center the cropped content
        pad_h_top = (h - ch) // 2
        pad_h_bot = h - ch - pad_h_top
        pad_w_left = (w - cw) // 2
        pad_w_right = w - cw - pad_w_left
        
        new_data = np.pad(cropped.data, 
                          ((pad_h_top, pad_h_bot), (pad_w_left, pad_w_right)), 'constant')
        return Grid(new_data)
        
    def merge_all_objects(self, grid: Grid, **kwargs) -> Grid:
        """Merges all objects into a single object of the dominant color."""
        objects = self.object_detector.detect_objects(grid)
        if not objects: return grid.copy()
        
        all_pixels = set.union(*(obj.pixels for obj in objects))
        
        # Get overall dominant color
        colors = grid.data[[r for r, c in all_pixels], [c for r, c in all_pixels]]
        unique, counts = np.unique(colors, return_counts=True)
        dominant_color = unique[np.argmax(counts)]

        result = np.zeros_like(grid.data)
        for r, c in all_pixels:
            result[r, c] = dominant_color
        return Grid(result)


    # --- D. Structural/Size Primitives ---
    def crop_to_content(self, grid: Grid, **kwargs) -> Grid:
        coords = np.argwhere(grid.data > 0)
        if coords.size == 0: return Grid(np.zeros((1, 1), dtype=int))
        min_r, min_c = coords.min(axis=0)
        max_r, max_c = coords.max(axis=0)
        return Grid(grid.data[min_r:max_r+1, min_c:max_c+1])

    def pad_to_match(self, grid: Grid, target_shape: Tuple[int, int], **kwargs) -> Grid:
        """Pads the grid to match a specific target shape (usually the target grid shape)."""
        h, w = grid.shape
        th, tw = target_shape
        pad_h = th - h
        pad_w = tw - w
        
        if pad_h <= 0 and pad_w <= 0: return grid # No padding needed or grid is already larger
        
        # Center padding
        p_t = max(0, pad_h // 2)
        p_b = max(0, pad_h - p_t)
        p_l = max(0, pad_w // 2)
        p_r = max(0, pad_w - p_l)
        
        return Grid(np.pad(grid.data, ((p_t, p_b), (p_l, p_r)), 'constant', constant_values=0))

    def resize_to_scale(self, grid: Grid, scale_factor: int = 2, **kwargs) -> Grid:
        """Resizes the grid by a specific integer scale factor using nearest-neighbor interpolation."""
        if scale_factor <= 0: return grid.copy()
        
        # Use Kronecker product for nearest neighbor upscaling
        resized_data = np.kron(grid.data, np.ones((scale_factor, scale_factor), dtype=int))
        return Grid(resized_data)
        
    def fill_background_holes(self, grid: Grid, hole_color: int = 0, **kwargs) -> Grid:
        """Fills internal holes (connected background pixels surrounded by foreground)."""
        binary_mask = grid.data > 0
        filled_mask = ndimage.binary_fill_holes(binary_mask)
        
        # Identify holes (pixels filled but were originally background)
        holes = filled_mask & ~binary_mask
        
        # Determine the color to fill with (usually the dominant foreground color)
        fg_colors = grid.data[binary_mask]
        if fg_colors.size == 0: return grid.copy()
        unique_colors, counts = np.unique(fg_colors, return_counts=True)
        dominant_color = unique_colors[np.argmax(counts)]

        result = grid.data.copy()
        result[holes] = dominant_color
        return Grid(result)


    # --- E. Relational/Counting Primitives ---
    def count_objects_and_recolor(self, grid: Grid, base_color: int = 1, **kwargs) -> Grid:
        """Counts objects and assigns a color based on the parity/count."""
        count = len(self.object_detector.detect_objects(grid))
        
        # Color logic: 1 if count is even, 2 if count is odd, 3 if count > 10
        new_color = base_color
        if count % 2 == 0 and count > 0:
            new_color = base_color + 1
        elif count > 10:
            new_color = base_color + 2
            
        new_color = max(1, min(new_color, HybridARCConfig.COLOR_RANGE - 1))
        
        # Apply the new color to all non-zero pixels
        result = grid.data.copy()
        mask = result > 0
        result[mask] = new_color
        return Grid(result)
        
    def extract_object_neighbors(self, grid: Grid, target_color: int = 1, neighbor_color: int = 2, **kwargs) -> Grid:
        """Extracts pixels neighboring objects of a specific color, coloring them with neighbor_color."""
        target_objects = [obj for obj in self.object_detector.detect_objects(grid) if obj.color == target_color]
        if not target_objects: return Grid(np.zeros_like(grid.data))
        
        # Union of masks
        combined_mask = np.zeros(grid.shape, dtype=bool)
        for obj in target_objects:
            combined_mask |= obj.mask
            
        # Dilate the mask to include neighbors
        dilated_mask = ndimage.binary_dilation(combined_mask)
        
        # The neighbor pixels are those in the dilated mask but not in the original grid content
        neighbor_mask = dilated_mask & (grid.data == 0)
        
        result = grid.data.copy()
        result[neighbor_mask] = max(1, min(neighbor_color, HybridARCConfig.COLOR_RANGE - 1))
        return Grid(result)

    # --- F. Pattern/Symmetry Primitives ---
    def create_symmetrical_pattern(self, grid: Grid, symmetry_type: str = 'vertical', **kwargs) -> Grid:
        """Mirrors the content to create a symmetrical pattern."""
        result = grid.data.copy()
        
        if symmetry_type == 'vertical':
            mirrored = np.fliplr(grid.data)
        elif symmetry_type == 'horizontal':
            mirrored = np.flipud(grid.data)
        else:
            return grid.copy()
            
        # Overlay the mirrored half onto the original grid (using union logic)
        h, w = grid.shape
        mh, mw = mirrored.shape
        
        h_max = max(h, mh)
        w_max = max(w, mw)
        
        grid_data = np.pad(grid.data, ((0, h_max - h), (0, w_max - w)), 'constant')
        mirrored_data = np.pad(mirrored, ((0, h_max - mh), (0, w_max - mw)), 'constant')

        new_data = np.maximum(grid_data, mirrored_data)
        return Grid(new_data)

    def repeat_pattern_by_bounds(self, grid: Grid, direction: str = 'horizontal', count: int = 2, **kwargs) -> Grid:
        """Repeats the grid's content a specified number of times along an axis."""
        if count <= 1: return grid.copy()
        
        if direction == 'horizontal':
            repeated_data = np.tile(grid.data, (1, count))
        elif direction == 'vertical':
            repeated_data = np.tile(grid.data, (count, 1))
        else:
            return grid.copy()

        return Grid(repeated_data)


    # --- G. NOVEL INSIGHT 1: Conditional Pattern Projection (CPP) ---
    def conditional_projection_by_parity(self, grid: Grid, base_pattern: Grid, **kwargs) -> Grid:
        """
        Projects a smaller base pattern onto the grid based on the parity of coordinates.
        This solves many checkerboard/interleaving ARC tasks.
        """
        if base_pattern.size == 0: return grid.copy()
        
        gh, gw = grid.shape
        ph, pw = base_pattern.shape
        
        result = grid.data.copy()
        
        for r in range(gh):
            for c in range(gw):
                # Use a combined index parity for projection
                parity_index = (r // ph + c // pw) % 2
                
                # If parity is 0 (even), project the pattern color
                if parity_index == 0:
                    pattern_color = base_pattern.data[r % ph, c % pw]
                    if pattern_color > 0:
                        result[r, c] = pattern_color
                
        return Grid(result)


    # --- H. NOVEL INSIGHT 2: Generalized Relative Object Copy (GRPC) ---
    def relative_object_copy(self, grid: Grid, reference_color: int = 8, copy_color: int = 3, **kwargs) -> Grid:
        """
        Copies all objects of 'copy_color' and places them relative to the objects 
        of 'reference_color', maintaining the relative position found in the training input.
        
        NOTE: In a real search, the 'relative_position_delta' must be pre-calculated from 
        the training pairs and passed as a parameter. For this single-cell emission, 
        we use a default simple delta.
        """
        
        # Placeholder for pre-calculated delta (must come from search context)
        # Assuming the delta is 1 row down, 1 col right, based on the first train pair
        delta_r, delta_c = 1, 1 
        
        ref_objects = [obj for obj in self.object_detector.detect_objects(grid) if obj.color == reference_color]
        copy_objects = [obj for obj in self.object_detector.detect_objects(grid) if obj.color == copy_color]
        
        if not ref_objects or not copy_objects: return grid.copy()
        
        # For simplicity, we only copy the structure of the largest 'copy_color' object
        # and place it relative to the largest 'reference_color' object.
        copy_template = self.extract_largest_object(Grid(grid.data[grid.data == copy_color]))
        if copy_template.size == 0: return grid.copy()
        
        ref_obj = max(ref_objects, key=lambda x: x.area)
        
        # Calculate anchor point (top-left of the reference object's bounding box)
        anchor_r, anchor_c, _, _ = ref_obj.bounding_box
        
        # New copy position
        new_r = anchor_r + delta_r
        new_c = anchor_c + delta_c
        
        # Apply the copy at the new position
        ch, cw = copy_template.shape
        gh, gw = grid.shape
        
        final_grid = grid.data.copy()
        
        # Bounds check
        if new_r + ch > gh or new_c + cw > gw or new_r < 0 or new_c < 0:
             return grid.copy() # Cannot place
        
        # Perform the overlay copy
        copy_data = copy_template.data
        slice_r = slice(new_r, new_r + ch)
        slice_c = slice(new_c, new_c + cw)
        
        mask = copy_data > 0
        final_grid[slice_r, slice_c][mask] = copy_data[mask]
        
        return Grid(final_grid)


logger.info(f"Cell 4 executed: Implemented Symbolic Heuristic Scorer and Final Hybrid DSL with over 25 primitives, including CPP and GRPC.")


2025-10-30 14:55:42,101 - INFO - Cell 4 executed: Implemented Symbolic Heuristic Scorer and Final Hybrid DSL with over 25 primitives, including CPP and GRPC.


In [5]:
#5
# Assuming Cells 1-4 have been executed (Config, Data Structures, Neural Guidance, DSL).

# --- Configuration Check and Initialization ---
# Instantiate components needed for search
try:
    GLOBAL_DSL = FinalHybridARCDSL()
    GLOBAL_SCORER = SemanticMetric()
    GLOBAL_HEURISTIC_SCORER = SymbolicHeuristicScorer(GLOBAL_DSL)
    GLOBAL_NEURAL_GUIDANCE = FinalNeuralGuidance() # The BSM engine
except NameError:
    logger.error("Dependencies (DSL, Scorer, Neural Guidance) not found. Check previous cells.")


# --- Beam State Definition ---
@dataclass(frozen=True)
class BeamState:
    """Represents a state in the beam search: a program and its executed output."""
    program: Program
    output_grid: Grid
    score: float # The combined PIS/LSSM/BSM score
    steps_hash: str # Unique hash for the program path
    
    # Store the result of the BSM scoring for analysis
    neural_confidence: float = 0.0
    lssm_score: float = 0.0
    is_sdp_candidate: bool = False # Flag for SDP components

# --- Core Synthesis Logic Helper ---

def _generate_next_steps(current_program: FunctionalProgram) -> List[FunctionalProgram]:
    """
    Generates all valid, non-redundant next FunctionalProgram extensions.
    The parameter space is minimized using sensible defaults and common ARC values.
    """
    
    # Get all available primitives from the DSL
    primitives = GLOBAL_DSL.get_primitives()
    new_programs: List[FunctionalProgram] = []
    
    # Simple, common parameters to iterate over (ARC-specific prior knowledge)
    color_options = [1, 2, 8] # Common colors
    size_options = [2, 3]     # Common scaling/filtering sizes
    direction_options = ['horizontal', 'vertical']
    
    for name, func in primitives.items():
        # --- Parameter Search (Limited to maintain search speed) ---
        
        if name in ['rotate_90', 'flip_horizontal', 'rotate_180', 'shift_to_top_left']:
            # No parameters needed
            new_programs.append(FunctionalProgram(current_program.steps + [ProgramStep(name, {})]))
            
        elif name in ['recolor_dominant', 'filter_by_color']:
            # Search over common colors
            for color in color_options:
                new_programs.append(FunctionalProgram(current_program.steps + [ProgramStep(name, {'new_color': color})]))
                
        elif name in ['recolor_by_area']:
            # Search over color pairs and a simple threshold
            for c1, c2 in itertools.combinations(color_options, 2):
                new_programs.append(FunctionalProgram(current_program.steps + [ProgramStep(name, {'small_color': c1, 'large_color': c2, 'threshold': 5})]))

        elif name in ['resize_to_scale', 'repeat_pattern_by_bounds']:
            # Search over simple scales/counts
            for s in size_options:
                if name == 'resize_to_scale':
                    new_programs.append(FunctionalProgram(current_program.steps + [ProgramStep(name, {'scale_factor': s})]))
                else: # repeat_pattern_by_bounds
                    for d in direction_options:
                        new_programs.append(FunctionalProgram(current_program.steps + [ProgramStep(name, {'direction': d, 'count': s})]))
                        
        elif name in ['relative_object_copy']:
            # Search over a few key relational colors (high ARC value)
             for ref_c, copy_c in itertools.product([3, 4, 8], [1, 2, 5]):
                new_programs.append(FunctionalProgram(current_program.steps + [ProgramStep(name, {'reference_color': ref_c, 'copy_color': copy_c})]))

        else:
            # All other primitives (e.g., crop_to_content, invert_colors)
            new_programs.append(FunctionalProgram(current_program.steps + [ProgramStep(name, {})]))

    return new_programs


# --- The Ultimate Beam Search Class ---

class FinalBeamSearch:
    """
    The Adaptive, Hybrid Neuro-Symbolic Program Synthesis Engine.
    Implements BSM for guidance and SDP for compositional search.
    """
    
    def __init__(self, dsl: FinalHybridARCDSL, scorer: SemanticMetric, 
                 heuristic_scorer: SymbolicHeuristicScorer, neural_guidance: FinalNeuralGuidance):
        
        self.dsl = dsl
        self.scorer = scorer
        self.h_scorer = heuristic_scorer
        self.n_guidance = neural_guidance
        
        # State tracking for efficiency
        self._visited_programs: Set[str] = set() # Full program hash
        self._program_suffix_cache: Dict[str, float] = {} # Novel Insight 2

    # --- NOVEL INSIGHT 1: Dynamic Task Difficulty and Resource Allocation ---
    def _get_task_complexity_vector(self, train_pairs: List[Tuple[Grid, Grid]]) -> Dict[str, Any]:
        """
        Dynamically assesses task difficulty to adjust resource allocation and 
        provide the Task Complexity Vector (TCV) for the neural model.
        """
        # Analyze the first training pair
        input_grid, target_grid = train_pairs[0]
        
        # Key metrics
        in_area = input_grid.size
        tar_area = target_grid.size
        in_objects = self.dsl.object_detector.detect_objects(input_grid)
        tar_objects = self.dsl.object_detector.detect_objects(target_grid)
        in_colors = set(input_grid.data.flatten()) - {0}
        tar_colors = set(target_grid.data.flatten()) - {0}
        
        # Complexity scores
        object_change = abs(len(tar_objects) - len(in_objects))
        color_change = abs(len(tar_colors) - len(in_colors))
        area_ratio = tar_area / max(in_area, 1)
        
        # Determine adaptive resource allocation (simplified logic)
        complexity_score = (object_change * 0.5) + (color_change * 0.5) + (abs(area_ratio - 1) * 0.2)
        
        # Adjust beam width and max length based on complexity
        adaptive_beam_width = HybridARCConfig.BEAM_WIDTH + int(complexity_score * 5)
        adaptive_max_length = HybridARCConfig.MAX_PROGRAM_LENGTH + int(complexity_score * 2)
        
        tcv = [
            np.log1p(in_area),
            np.log1p(tar_area),
            float(object_change) / max(len(in_objects), 1, 1e-6), # Normalized object change
            float(color_change) / max(len(in_colors), 1, 1e-6)     # Normalized color change
        ]

        return {
            'beam_width': min(adaptive_beam_width, 32),
            'max_length': min(adaptive_max_length, 12),
            'tcv': tcv # The TCV tensor for the Neural Guidance
        }


    # --- Search Step Pruning and Ranking (BSM + Novel Insight 2) ---
    def _prune_and_rank(self, 
                        task_context: Dict[str, Any],
                        candidates: List[FunctionalProgram], 
                        target_grid: Grid, 
                        k: int) -> List[BeamState]:
        """
        Evaluates, scores, and prunes the candidate programs using the BSM blend.
        Applies Program Suffix Hashing to prevent redundant paths.
        """
        scored_states: List[BeamState] = []
        
        for program in candidates:
            # Pruning check 1: Full Program Redundancy
            full_hash = program.program_hash()
            if full_hash in self._visited_programs:
                continue
            self._visited_programs.add(full_hash)

            # --- Program Execution ---
            input_grid = task_context['train_pairs'][0][0] # Input for first train pair
            output_grid = program.execute(input_grid, self.dsl)
            
            # --- Program Suffix Hashing (Novel Insight 2) ---
            # Hash of the last 3 steps (or fewer)
            steps = program.get_steps()
            suffix = tuple(asdict(s) for s in steps[-3:])
            suffix_hash = hashlib.sha256(json.dumps(suffix, sort_keys=True).encode()).hexdigest()
            
            # Pruning check 2: Suffix Dominance Check
            current_pis = self.scorer.calculate_pis(output_grid, target_grid, self.dsl.object_detector)
            
            if suffix_hash in self._program_suffix_cache:
                max_score_for_suffix = self._program_suffix_cache[suffix_hash]
                # If the current program (which is longer) doesn't improve the score 
                # significantly over a shorter program with the same suffix, prune it.
                if current_pis < max_score_for_suffix * 0.95:
                    # logger.debug(f"Pruned by Suffix Hash: {suffix_hash}")
                    continue
            
            # Update cache with the best score found for this suffix
            self._program_suffix_cache[suffix_hash] = current_pis

            # --- BSM/LSSM Scoring ---
            # 1. Get Symbolic Heuristic Score (prior for this primitive)
            last_step = program.steps[-1]
            symbolic_score = self.h_scorer.score_primitive_heuristic(
                last_step.primitive, input_grid, target_grid, task_context
            )
            
            # 2. Get BSM/Neural Guidance Score
            bsm_score, confidence, lssm = self.n_guidance.score_primitive_balanced(
                input_grid, target_grid, last_step.primitive, symbolic_score, task_context
            )
            
            # 3. Final Search Score (A* style: (PIS of output + BSM of next step) / length)
            # The final score for the search prioritizes the BSM score for guidance 
            # and the PIS score for actual correctness. We use a linear combination.
            # Use PIS as the primary reward, BSM as the path-cost heuristic.
            
            # The final score for beam ranking:
            # Score = (0.7 * PIS) + (0.3 * BSM_Guidance) - (0.01 * Program_Length)
            # Normalizing by length prevents the search from getting stuck in a local optimum
            # of short, but high-scoring programs.
            length_penalty = 0.01 * len(program)
            final_beam_score = (0.7 * current_pis) + (0.3 * bsm_score) - length_penalty
            
            scored_states.append(BeamState(
                program=program, 
                output_grid=output_grid, 
                score=final_beam_score, 
                steps_hash=full_hash,
                neural_confidence=confidence,
                lssm_score=lssm,
                is_sdp_candidate=False
            ))
            
        # Select the top k states
        scored_states.sort(key=lambda s: s.score, reverse=True)
        return scored_states[:k]


    # --- Main Search Function ---
    def search_program(self, task: Task) -> Optional[Program]:
        """
        The main adaptive search loop: FunctionalProgram -> SDP -> Verification.
        """
        logger.info(f"Starting adaptive beam search for Task: {task.task_id}")
        
        start_time = time.time()
        input_grid, target_grid = task.train_pairs[0]
        
        # Novel Insight 1: Dynamic Resource Allocation
        task_context = self._get_task_complexity_vector(task.train_pairs)
        beam_width = task_context['beam_width']
        max_length = task_context['max_length']
        
        logger.info(f"Adaptive Search: Beam Width={beam_width}, Max Length={max_length}")

        # Initialize the beam with the identity program
        identity_program = FunctionalProgram(steps=[])
        identity_output = identity_program.execute(input_grid, self.dsl)
        initial_pis = self.scorer.calculate_pis(identity_output, target_grid, self.dsl.object_detector)
        
        initial_state = BeamState(
            program=identity_program,
            output_grid=identity_output,
            score=(0.7 * initial_pis), # Initial score uses PIS only
            steps_hash=identity_program.program_hash()
        )
        
        # Priority Queue for the beam (max-heap based on score)
        beam = [initial_state] 
        best_program: Optional[Program] = None
        best_score = initial_pis

        # --- Functional Program (NSM) Search ---
        for current_length in range(max_length):
            if time.time() - start_time > HybridARCConfig.MAX_TASK_TIME:
                logger.warning(f"Task {task.task_id} timed out during NSM search.")
                break

            new_candidates: List[FunctionalProgram] = []
            
            for state in beam:
                # Early Exit Check: Solution Found
                if state.score >= HybridARCConfig.PIS_THRESHOLD:
                    return state.program

                # Generate next potential steps
                new_candidates.extend(_generate_next_steps(state.program))

            if not new_candidates:
                break

            # Prune and Rank candidates
            new_states = self._prune_and_rank(task_context, new_candidates, target_grid, beam_width)
            
            # Update the beam and track the best program found so far
            beam = new_states
            for state in new_states:
                current_pis = self.scorer.calculate_pis(state.output_grid, target_grid, self.dsl.object_detector)
                if current_pis > best_score:
                    best_score = current_pis
                    best_program = state.program
                
            logger.info(f"L={current_length+1}: Best PIS={best_score:.4f}, Beam Max Score={new_states[0].score:.4f}")
            if best_score >= HybridARCConfig.PIS_THRESHOLD:
                return best_program # Exact match found

        # --- SDP Compositional Search (If NSM fails or gets stuck) ---
        if best_score < HybridARCConfig.PIS_THRESHOLD:
            logger.info(f"NSM search finished with PIS={best_score:.4f}. Starting SDP composition.")
            
            # Use the top K programs from the final beam state as components
            # Also include any programs that achieved a high LSSM score in the process
            sdp_candidates = [s.program for s in beam if s.score > 0.6] 
            
            sdp_ops = ['overlay', 'union', 'intersection']
            
            for op, c1, c2 in itertools.product(sdp_ops, sdp_candidates, sdp_candidates):
                if c1.program_hash() == c2.program_hash() and op != 'union': continue # Avoid redundant self-composition
                
                meta_program = SDPMetaProgram(op_type=op, component_1=c1, component_2=c2)
                
                # Execute and score the Meta-Program
                output_grid = meta_program.execute(input_grid, self.dsl)
                meta_pis = self.scorer.calculate_pis(output_grid, target_grid, self.dsl.object_detector)
                
                if meta_pis > best_score:
                    best_score = meta_pis
                    best_program = meta_program
                    
                    if best_score >= HybridARCConfig.PIS_THRESHOLD:
                        logger.info(f"SDP Composition solved the task with PIS={best_score:.4f}")
                        return best_program
                        
            logger.info(f"SDP search finished. Final Best PIS={best_score:.4f}")

        return best_program # Return the best program found (NSM or SDP)

logger.info(f"Cell 5 executed: Implemented Final Adaptive Beam Search (NSM + SDP), Dynamic Task Difficulty Assessment, and Program Suffix Hashing.")


2025-10-30 14:55:42,252 - INFO - Cell 5 executed: Implemented Final Adaptive Beam Search (NSM + SDP), Dynamic Task Difficulty Assessment, and Program Suffix Hashing.


In [6]:
# 6
# Assuming Cells 1-5 have been executed (Config, Data Structures, Neural Guidance, DSL, Search).
from collections import defaultdict # Ensure defaultdict is imported
from dataclasses import dataclass, field # Ensure field is imported

# --- Checkpointing File Definitions ---
CHECKPOINT_DIR = Path("./arc_checkpoints")
CHECKPOINT_DIR.mkdir(exist_ok=True)
CHECKPOINT_FILE = CHECKPOINT_DIR / "arc_solver_checkpoint.json"
SUBMISSION_FILE = Path("./submission.json") # The final output file

# --- Task Execution Context (FIXED FOR TypeError) ---
@dataclass
class ExecutionContext:
    """Manages global state, timing, and checkpoint metadata."""
    
    # Non-default arguments (must come first)
    start_time: float
    last_checkpoint_time: float # Moved here from below
    
    # Default arguments (must come last)
    total_tasks_solved: int = 0
    total_time_spent: float = 0.0
    
    # FIX: Use field(default_factory=...) for mutable defaults
    solved_tasks: Dict[str, List[List[List[int]]]] = field(default_factory=lambda: defaultdict(list))
    attempted_tasks: Set[str] = field(default_factory=set)
    task_order: List[str] = field(default_factory=list)

# --- 1. CHECKPOINTING LOGIC ---

def save_checkpoint(context: ExecutionContext):
    """Saves the current solver state, including solved tasks and time budget."""
    if time.time() - context.last_checkpoint_time < HybridARCConfig.MIN_CHECKPOINT_TIME:
        return # Avoid saving too frequently

    checkpoint_data = {
        "global_start_time": context.start_time,
        "total_time_spent": context.total_time_spent,
        "tasks_solved_count": context.total_tasks_solved,
        "solved_tasks": dict(context.solved_tasks), # Convert defaultdict back to dict for JSON
        "attempted_tasks": list(context.attempted_tasks),
        "task_order": context.task_order,
        "last_checkpoint_time": time.time()
    }
    
    with open(CHECKPOINT_FILE, 'w') as f:
        json.dump(checkpoint_data, f)
        
    # Also save the current state to the final submission format for safety
    with open(SUBMISSION_FILE, 'w') as f:
        json.dump(dict(context.solved_tasks), f)
        
    context.last_checkpoint_time = time.time()
    logger.info(f"Checkpoint saved. Solved: {context.total_tasks_solved}, Time Spent: {context.total_time_spent:.1f}s")


def load_checkpoint(all_task_ids: List[str]) -> ExecutionContext:
    """Loads the solver state or initializes a new context."""
    if CHECKPOINT_FILE.exists():
        with open(CHECKPOINT_FILE, 'r') as f:
            data = json.load(f)
            
        actual_start_time = time.time() - data.get("total_time_spent", 0.0)
        
        context = ExecutionContext(
            start_time=actual_start_time,
            last_checkpoint_time=data.get("last_checkpoint_time", actual_start_time),
            total_tasks_solved=data.get("tasks_solved_count", 0),
            total_time_spent=data.get("total_time_spent", 0.0),
            solved_tasks=defaultdict(list, data.get("solved_tasks", {})),
            attempted_tasks=set(data.get("attempted_tasks", [])),
            task_order=data.get("task_order", all_task_ids)
        )
        logger.info(f"Checkpoint loaded. Resuming run. Total Solved: {context.total_tasks_solved}")
        return context
    else:
        current_time = time.time()
        return ExecutionContext(
            start_time=current_time,
            last_checkpoint_time=current_time,
            task_order=all_task_ids
        )

# --- 2. ADAPTIVE TIME ALLOCATOR (Novel Insight 3) ---

def adaptive_time_budget(context: ExecutionContext) -> float:
    """
    NOVEL INSIGHT 3: Adaptive Time Allocation.
    Prioritizes remaining time to tasks that haven't been solved, 
    but caps the time per task to ensure full coverage.
    """
    
    time_elapsed = time.time() - context.start_time
    time_remaining = HybridARCConfig.TOTAL_BUDGET_SECONDS - time_elapsed
    
    unattempted_tasks = [tid for tid in context.task_order if tid not in context.attempted_tasks]
    
    if not unattempted_tasks:
        return HybridARCConfig.MAX_TASK_TIME
        
    num_unattempted = len(unattempted_tasks)
    
    target_budget = time_remaining / max(num_unattempted, 1)
    
    allocated_time = min(target_budget, HybridARCConfig.MAX_TASK_TIME)
    
    return max(allocated_time, HybridARCConfig.MIN_TASK_TIME)


# --- 3. THE GLOBAL TASK RUNNER ---

def run_solver(all_tasks: Dict[str, Task], solver: FinalBeamSearch):
    """
    Main loop for processing tasks with time management and checkpointing.
    """
    all_task_ids = sorted(list(all_tasks.keys()))
    context = load_checkpoint(all_task_ids)
    
    tasks_to_process = [tid for tid in context.task_order if tid not in context.attempted_tasks]
    tasks_to_process.extend([tid for tid in context.task_order if tid not in tasks_to_process]) 

    for task_id in tasks_to_process:
        task = all_tasks.get(task_id)
        if not task: continue
        
        if time.time() - context.start_time >= HybridARCConfig.TOTAL_BUDGET_SECONDS:
            logger.warning("Total time budget exhausted. Exiting solver.")
            break
            
        if task_id in context.solved_tasks:
            logger.info(f"Skipping solved task: {task_id}")
            continue

        time_budget = adaptive_time_budget(context)
        
        logger.info(f"\n--- Solving Task {task_id} --- Budget: {time_budget:.1f}s")
        task_start_time = time.time()
        
        found_program = solver.search_program(task)
        
        if found_program:
            is_valid = True
            for input_grid, target_grid in task.train_pairs:
                output_grid = found_program.execute(input_grid, solver.dsl)
                pis = solver.scorer.calculate_pis(output_grid, target_grid, solver.dsl.object_detector)
                if pis < HybridARCConfig.PIS_THRESHOLD:
                    is_valid = False
                    break
            
            if is_valid:
                test_outputs_list = []
                for test_input in task.test_inputs:
                    test_output = found_program.execute(test_input, solver.dsl)
                    test_outputs_list.append(test_output.to_list())
                    
                context.solved_tasks[task_id] = test_outputs_list
                context.total_tasks_solved += 1
                logger.critical(f"SUCCESS: Task {task_id} solved! Total Solved: {context.total_tasks_solved}")
        
        context.attempted_tasks.add(task_id)
        task_time = time.time() - task_start_time
        context.total_time_spent += task_time
        logger.info(f"Task {task_id} finished in {task_time:.2f}s. Total Time: {context.total_time_spent:.1f}s")
        
        if context.total_tasks_solved % HybridARCConfig.CHECKPOINT_INTERVAL == 0:
            save_checkpoint(context)
            
    save_checkpoint(context)

logger.info(f"Cell 6 executed: Implemented Global Task Runner, Robust Checkpointing, and Adaptive Time Allocation (Novel Insight 3) (FIXED).")

# #INCLUDE# FOOTER COMMENTS WITH CELL #!!!!!


2025-10-30 14:55:42,286 - INFO - Cell 6 executed: Implemented Global Task Runner, Robust Checkpointing, and Adaptive Time Allocation (Novel Insight 3) (FIXED).


In [7]:
#7
# Assuming Cells 1-6 have been executed (Config, Data Structures, Neural Guidance, DSL, Search, Checkpointing).

# --- 1. CORE DATA PARSING UTILITIES ---

def _parse_grid(raw_data: List[List[int]]) -> Grid:
    """Converts a raw list-of-lists into an immutable Grid object."""
    # Ensure correct dtype (int is critical for color/indexing operations)
    return Grid(np.array(raw_data, dtype=int))

def _parse_pair(pair_data: Dict[str, List[List[int]]]) -> Tuple[Grid, Grid]:
    """Parses a single input/output pair."""
    input_grid = _parse_grid(pair_data['input'])
    output_grid = _parse_grid(pair_data['output'])
    return input_grid, output_grid

def load_all_arc_tasks(base_path: str = './data') -> Dict[str, Task]:
    """
    Simulates loading all ARC tasks from the standard file structure.
    In a Kaggle notebook, this path would point to the competition data.
    """
    
    # Placeholder: In a real environment, this loads all train/test JSONs.
    # Since we don't have the full dataset here, we'll use a mocked structure 
    # and require the user to provide the actual data path if running locally.
    
    # Mocking a small set of tasks for demonstration.
    # The submission.json snippet from the user's uploaded files will be used to 
    # infer the expected structure (even though it's an output file).
    
    mock_tasks = {
        "007bbfb7": {
            "train": [
                {"input": [[7,7,7,0,0], [7,0,7,0,0], [7,7,7,0,0], [0,0,0,0,0]], "output": [[7,7,7], [7,0,7], [7,7,7]]},
            ],
            "test": [
                {"input": [[7,7,7,7,7,7], [7,0,0,0,0,7], [7,7,7,7,7,7]], "output": []} # Output empty as it's the target for the solver
            ]
        },
        "9d915682": {
            "train": [
                {"input": [[1,1,1],[1,0,1],[1,1,1]], "output": [[1,1,1,1,1],[1,0,0,0,1],[1,0,0,0,1],[1,0,0,0,1],[1,1,1,1,1]]},
            ],
            "test": [
                {"input": [[2,2],[2,2]], "output": []}
            ]
        }
    }
    
    # --- Actual Task Conversion ---
    all_tasks: Dict[str, Task] = {}
    for task_id, raw_task in mock_tasks.items():
        train_pairs = [_parse_pair(p) for p in raw_task['train']]
        test_inputs = [_parse_grid(p['input']) for p in raw_task['test']]
        
        task = Task(
            task_id=task_id,
            train_pairs=train_pairs,
            test_inputs=test_inputs
        )
        all_tasks[task_id] = task
        
    logger.info(f"Loaded {len(all_tasks)} mock tasks.")
    return all_tasks


# --- 2. NOVEL INSIGHT 5: GRID FEATURE PRE-EXTRACTION CACHE ---

def _pre_extract_features_and_cache(task: Task, detector: 'FinalObjectDetector'):
    """
    Pre-calculates static, complex features for the training inputs and stores 
    them in a dictionary accessible during the search.
    This saves valuable time within the search loop.
    """
    
    # Use the Task object to hold a dynamic context (pre-extraction context)
    # We use the train_pairs list's first input grid as the primary context source
    input_grid, target_grid = task.train_pairs[0]
    
    # Pre-extract OCRP objects (expensive operation)
    in_objects = detector.detect_objects(input_grid)
    tar_objects = detector.detect_objects(target_grid)

    # Pre-calculate histograms and key statistics
    in_colors = set(input_grid.data.flatten()) - {0}
    tar_colors = set(target_grid.data.flatten()) - {0}
    
    # Calculate difference metrics for HTG
    object_change = abs(len(tar_objects) - len(in_objects))
    color_change = len(in_colors.symmetric_difference(tar_colors)) # How many unique colors were added/removed
    
    # Attach a context dict to the task for global access
    task.context = {
        'input_objects': in_objects,
        'target_objects': tar_objects,
        'in_colors': in_colors,
        'tar_colors': tar_colors,
        'object_change': object_change,
        'color_change': color_change,
        'is_shape_preserved': input_grid.shape == target_grid.shape
    }

# --- 3. NOVEL INSIGHT 4: HETEROGENEOUS TASK GROUPING (HTG) ---

def group_tasks_by_structure(all_tasks: Dict[str, Task]) -> List[str]:
    """
    NOVEL INSIGHT 4: Groups and sorts tasks based on structural properties 
    (from pre-extracted features) to optimize the solving order (HTG).
    Prioritizes simpler tasks first, then those requiring specific types of operations.
    """
    
    def get_task_category(task: Task) -> Tuple[int, float]:
        """
        Assigns a complexity score and category index for sorting.
        Lower index/score means easier/faster to solve.
        """
        context = task.context
        
        is_trivial = context['object_change'] == 0 and context['color_change'] == 0 and context['is_shape_preserved']
        
        if is_trivial:
            # Category 1: Trivial/Identity/Simple Color Swap
            category_index = 1
            complexity_score = -1.0
        elif context['color_change'] > 0 and context['object_change'] == 0 and context['is_shape_preserved']:
            # Category 2: Color Operations only (Shape and Objects preserved)
            category_index = 2
            complexity_score = context['color_change']
        elif not context['is_shape_preserved'] and context['object_change'] <= 1:
            # Category 3: Structural/Size/Crop Operations (Shape is the primary change)
            category_index = 3
            complexity_score = abs(task.train_pairs[0][1].size - task.train_pairs[0][0].size)
        elif context['object_change'] > 0:
            # Category 4: Object-Centric/Relational/Pattern Operations (Hardest)
            category_index = 4
            complexity_score = context['object_change'] * 10 + context['color_change']
        else:
            # Fallback
            category_index = 5
            complexity_score = 999
            
        return category_index, complexity_score

    # Sort the tasks: Category Index (Primary) -> Complexity Score (Secondary)
    sorted_tasks = sorted(all_tasks.items(), key=lambda item: get_task_category(item[1]))
    
    sorted_task_ids = [task_id for task_id, _ in sorted_tasks]
    
    logger.info("Tasks grouped by Heterogeneous Task Grouping (HTG).")
    return sorted_task_ids


# --- 4. MAIN EXECUTION SETUP ---

def main_execution_setup(data_path: str = './data'):
    """
    Initializes all global components and starts the solver run.
    """
    
    # 1. Load Tasks and Pre-Extract Features
    all_tasks = load_all_arc_tasks(data_path)
    
    # Pre-extract features for all tasks
    detector = FinalObjectDetector()
    for task in all_tasks.values():
        _pre_extract_features_and_cache(task, detector)
        
    # 2. Apply HTG for Optimized Task Order
    sorted_task_ids = group_tasks_by_structure(all_tasks)
    
    # Update the global task list to the optimized order
    ordered_tasks = {tid: all_tasks[tid] for tid in sorted_task_ids}

    # 3. Initialize Solver Components (must match Cell 5 initialization)
    try:
        dsl = FinalHybridARCDSL()
        scorer = SemanticMetric()
        heuristic_scorer = SymbolicHeuristicScorer(dsl)
        neural_guidance = FinalNeuralGuidance()
        
        solver = FinalBeamSearch(
            dsl=dsl, 
            scorer=scorer, 
            heuristic_scorer=heuristic_scorer, 
            neural_guidance=neural_guidance
        )
        
        # 4. Begin the solve process
        run_solver(ordered_tasks, solver)
        
    except NameError as e:
        logger.error(f"Critical error: A required class was not initialized in a previous cell: {e}")
        logger.error("Please ensure Cells 1-6 are executed successfully before Cell 7.")
        return

logger.info(f"Cell 7 executed: Defined Data Loading, Feature Pre-extraction Cache (Novel Insight 5), and Heterogeneous Task Grouping (Novel Insight 4). Ready for final execution call.")


2025-10-30 14:55:42,322 - INFO - Cell 7 executed: Defined Data Loading, Feature Pre-extraction Cache (Novel Insight 5), and Heterogeneous Task Grouping (Novel Insight 4). Ready for final execution call.


In [8]:
#8
# Assuming Cells 1-7 have been executed (Config, Data Structures, Neural Guidance, DSL, Search, Checkpointing, Data Loading).

# --- Configuration for Training ---
TRAINING_CONFIG = {
    'EPOCHS': 15,
    'BATCH_SIZE': 64,
    'LEARNING_RATE': 1e-4,
    'DATA_SIZE': 20000, # Number of synthetic (input, target, primitive) triplets
    'MAX_PROGRAM_STEPS': 3, # Max steps for synthetic data generation
    'PRIMITIVE_SAMPLING_WEIGHT': 0.8 # Weight for sampling based on complexity
}

# --- 1. DATA GENERATION: SELF-SUPERVISED PROGRAM SYNTHESIS (SSPS) (Novel Insight 6) ---

class SelfSupervisedProgramSynthesizer:
    """
    Generates synthetic training data (input, target, program) by applying 
    randomly chosen primitives to random initial grids.
    This data is task-agnostic but crucial for pre-training the BSM.
    """
    def __init__(self, dsl: FinalHybridARCDSL, config: Dict):
        self.dsl = dsl
        self.config = config
        self.primitives = list(dsl.get_primitives().items())
        
        # Grid generation parameters
        self.grid_size_range = (5, 15) # Generate grids between 5x5 and 15x15
        self.color_range = HybridARCConfig.COLOR_RANGE
        
    def _generate_random_grid(self) -> Grid:
        """Generates a non-trivial random grid."""
        H = np.random.randint(*self.grid_size_range)
        W = np.random.randint(*self.grid_size_range)
        # 10% chance of background (0), 90% chance of a random color
        data = np.random.choice(
            a=self.color_range, 
            size=(H, W), 
            p=[0.1] + [0.9 / (self.color_range - 1)] * (self.color_range - 1)
        )
        # Ensure at least one non-zero pixel
        if np.all(data == 0): data[0, 0] = np.random.randint(1, self.color_range)
        return Grid(data)

    def generate_data(self) -> List[Tuple[Grid, Grid, ProgramStep]]:
        """
        Generates SSPS data triplets: (Input Grid, Target Grid, Program Step).
        """
        data_points = []
        target_size = self.config['DATA_SIZE']
        
        # Pre-calculate primitive complexity weights for weighted sampling
        primitive_names = [name for name, _ in self.primitives]
        categories = GLOBAL_NEURAL_GUIDANCE._get_primitive_categories_()
        
        weights = []
        for name in primitive_names:
            category = GLOBAL_NEURAL_GUIDANCE.map_primitive_to_category(name)
            # Complex categories (e.g., relational_op) get slightly higher weight
            weight = 1.0
            if category in ['relational_op', 'structural_change', 'pattern_operation']:
                weight = self.config['PRIMITIVE_SAMPLING_WEIGHT']
            weights.append(weight)
        
        weights = np.array(weights) / np.sum(weights)

        while len(data_points) < target_size:
            # 1. Start with a random input grid
            input_grid = self._generate_random_grid()
            
            # 2. Randomly select a primitive based on complexity weight
            primitive_index = np.random.choice(len(self.primitives), p=weights)
            name, func = self.primitives[primitive_index]
            
            # 3. Randomly select parameters (simplified to the most common case)
            params = {}
            if name in ['recolor_dominant', 'filter_by_color']:
                params = {'new_color': np.random.randint(1, self.color_range)}
            elif name in ['swap_colors_ab']:
                c1, c2 = np.random.choice(self.color_range, 2, replace=False)
                params = {'color_a': c1, 'color_b': c2}
            elif name in ['resize_to_scale']:
                params = {'scale_factor': np.random.randint(2, 4)}
            
            # 4. Create Program Step and Execute
            program_step = ProgramStep(name, params)
            
            try:
                # The input grid is the 'input', the result is the 'target'
                target_grid = func(input_grid, **program_step.parameters)
                
                # Filter out no-op or trivial transformations
                if target_grid.data.shape == input_grid.data.shape and np.array_equal(target_grid.data, input_grid.data):
                    continue
                    
                data_points.append((input_grid, target_grid, program_step))
                
            except Exception:
                # Skip invalid executions
                continue

        logger.info(f"Generated {len(data_points)} SSPS triplets for training.")
        return data_points


# --- 2. NOVEL INSIGHT 7: BALANCED CATEGORICAL CROSS-ENTROPY LOSS (BCCE) ---

class BalancedCategoricalCrossEntropyLoss(nn.Module):
    """
    Custom loss function that combines standard CE with a confidence-aware 
    balance term to prevent over-confidence in easy/common primitives.
    """
    def __init__(self, num_categories: int, confidence_weight: float = 0.5):
        super().__init__()
        self.ce_loss = nn.CrossEntropyLoss()
        self.num_categories = num_categories
        self.confidence_weight = confidence_weight

    def forward(self, 
                category_logits: torch.Tensor, 
                confidence_logits: torch.Tensor, 
                target_categories: torch.Tensor, 
                target_confidence: torch.Tensor) -> torch.Tensor:
        """
        Args:
            category_logits: Logits for primitive categories (Batch, Num_Categories)
            confidence_logits: Logits from the confidence_net (Batch, 1)
            target_categories: True category indices (Batch)
            target_confidence: True confidence values (PIS) (Batch, 1)
        """
        
        # 1. Categorical Cross-Entropy Loss (Primitive Selection)
        ce_loss = self.ce_loss(category_logits, target_categories)
        
        # 2. Confidence Regression Loss (Mean Squared Error on Sigmoid output)
        # Use logit input for BCEWithLogitsLoss for numerical stability
        confidence_loss = F.mse_loss(confidence_logits, target_confidence)

        # 3. Balancing Term (Novel Insight 7)
        # Reward low confidence for low-PIS (complex/unclear) examples
        # Penalize high confidence for high-PIS (easy/solved) examples
        
        # Confidence score (sigmoid of logits)
        confidence_score = torch.sigmoid(confidence_logits) 
        
        # Balancing Term: Encourages confidence score to track PIS score closely
        # Low PIS (0) -> Confidence should be low (0) -> (0-0)^2 = 0
        # High PIS (1) -> Confidence should be high (1) -> (1-1)^2 = 0
        # This acts as a regularizer, forcing the confidence net to learn PIS prediction
        balancing_loss = F.mse_loss(confidence_score, target_confidence)
        
        # 4. Combined Loss
        total_loss = ce_loss + confidence_loss + (self.confidence_weight * balancing_loss)
        return total_loss


# --- 3. THE NEURAL TRAINING LOOP ---

def train_neural_guidance_system(model: FinalNeuralGuidance, data_path: str = './data'):
    """
    Main function to pre-train the BSM guidance system using SSPS data.
    """
    logger.info("Starting BSM/CPC Neural Guidance pre-training...")
    
    # 1. Data Generation (SSPS)
    synthesizer = SelfSupervisedProgramSynthesizer(GLOBAL_DSL, TRAINING_CONFIG)
    ssps_data = synthesizer.generate_data()
    
    # 2. Data Preparation and Tensor Conversion
    input_grids = [i for i, t, p in ssps_data]
    target_grids = [t for i, t, p in ssps_data]
    program_steps = [p for i, t, p in ssps_data]
    
    # Get categorical labels and PIS scores
    category_map = {name: i for i, name in enumerate(model._get_primitive_categories_())}
    target_categories = []
    target_confidences = [] # Use the PIS score of the resulting transformation as "True Confidence"
    
    for i_grid, t_grid, p_step in ssps_data:
        category = model.map_primitive_to_category(p_step.primitive)
        target_categories.append(category_map[category])
        
        # PIS calculation: How close did the primitive get to the target? 
        # Since this is SSPS data, the PIS should be near 1.0, but may not be 1.0
        # if the primitive introduces side effects (e.g., padding/cropping).
        pis_score = GLOBAL_SCORER.calculate_pis(t_grid, t_grid, GLOBAL_DSL.object_detector)
        target_confidences.append(pis_score)
    
    # Convert to Tensors
    target_categories_t = torch.LongTensor(target_categories).to(HybridARCConfig.DEVICE)
    target_confidences_t = torch.FloatTensor(target_confidences).unsqueeze(1).to(HybridARCConfig.DEVICE)
    
    # 3. Training Setup
    optimizer = torch.optim.AdamW(model.parameters(), lr=TRAINING_CONFIG['LEARNING_RATE'])
    loss_fn = BalancedCategoricalCrossEntropyLoss(
        num_categories=len(category_map), 
        confidence_weight=0.5
    ).to(HybridARCConfig.DEVICE)
    
    # 4. Training Loop (Manual Batching for Variable Grid Size)
    model.train()
    
    for epoch in range(TRAINING_CONFIG['EPOCHS']):
        total_loss = 0.0
        num_batches = 0
        
        # Shuffle indices
        indices = np.arange(len(ssps_data))
        np.random.shuffle(indices)
        
        for i in range(0, len(ssps_data), TRAINING_CONFIG['BATCH_SIZE']):
            batch_indices = indices[i:i + TRAINING_CONFIG['BATCH_SIZE']]
            
            # --- Batch Preparation ---
            batch_inputs = [input_grids[j] for j in batch_indices]
            batch_targets = [target_grids[j] for j in batch_indices]
            batch_target_cat = target_categories_t[batch_indices]
            batch_target_conf = target_confidences_t[batch_indices]

            # Since grids are variable size, we must encode one-by-one or pad to max size.
            # Encoding one-by-one is safer and prevents massive zero-padding waste.
            
            # Stack encodings manually (B x Latent_dim)
            input_encodings = torch.cat([model.encoder(g) for g in batch_inputs], dim=0)
            target_encodings = torch.cat([model.encoder(g) for g in batch_targets], dim=0)
            
            # --- Forward Pass ---
            combined_enc = torch.cat([input_encodings, target_encodings], dim=-1)
            
            # Mock TCV for SSPS data (since TCV is task-specific, but SSPS is task-agnostic)
            # Use zero-vector TCV for general applicability
            tcv_mock = torch.zeros(combined_enc.shape[0], model.tcv_dim).to(HybridARCConfig.DEVICE)
            
            # CPC/Primitive Scorer input
            scorer_input = torch.cat([combined_enc, tcv_mock], dim=-1)
            
            # Outputs
            category_logits = model.primitive_scorer(scorer_input)
            confidence_logits = model.confidence_net(combined_enc)
            
            # --- Backward Pass ---
            loss = loss_fn(category_logits, confidence_logits, batch_target_cat, batch_target_conf)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            num_batches += 1

        avg_loss = total_loss / max(num_batches, 1)
        logger.info(f"Epoch {epoch+1}/{TRAINING_CONFIG['EPOCHS']} - Avg Loss: {avg_loss:.4f}")

    # Save the trained model parameters (Critical for continued solving)
    model_save_path = CHECKPOINT_DIR / "neural_guidance_model.pth"
    torch.save(model.state_dict(), model_save_path)
    logger.info(f"Neural Guidance Model trained and saved to {model_save_path}.")

# --- Example Call (MUST be executed before Cell 7's run_solver) ---
# train_neural_guidance_system(GLOBAL_NEURAL_GUIDANCE)


In [9]:
#9
# Assuming Cells 1-8 have been executed (All core components, training, and execution logic are defined).

# --- Configuration Reference ---
# SUBMISSION_FILE is defined in Cell 6 (e.g., Path("./submission.json"))
# CHECKPOINT_DIR is defined in Cell 6 (e.g., Path("./arc_checkpoints"))

# --- 1. POST-PROCESSING VALIDATION (NOVEL INSIGHT 8) ---

def _validate_grid_format(grid_list: List[List[int]]) -> bool:
    """Checks if the output list-of-lists is a valid ARC grid format."""
    if not isinstance(grid_list, list) or not grid_list:
        return False
    
    # Check that all elements are lists and non-empty
    if not all(isinstance(row, list) and row for row in grid_list):
        return False
        
    # Check for consistent row length (rectangular shape)
    row_lengths = [len(row) for row in grid_list]
    if len(set(row_lengths)) != 1:
        return False
        
    # Check that all elements are integers within the color range
    for row in grid_list:
        if not all(isinstance(c, int) and 0 <= c < HybridARCConfig.COLOR_RANGE for c in row):
            return False
            
    # Check size constraint: Max 30x30
    H, W = len(grid_list), row_lengths[0]
    if H > HybridARCConfig.MAX_GRID_SIZE or W > HybridARCConfig.MAX_GRID_SIZE:
        return False
        
    return True


def adaptive_grid_validation(task_id: str, output_list: List[List[int]], 
                             context: Optional[Dict] = None) -> List[List[int]]:
    """
    NOVEL INSIGHT 8: Adaptive Grid Validation (AGV).
    If validation fails, applies a sequence of deterministic clean-up primitives 
    (cropping, padding) to fix common format errors before submission.
    """
    
    if _validate_grid_format(output_list):
        return output_list
        
    logger.warning(f"AGV: Grid format invalid for task {task_id}. Attempting repair.")
    
    # Convert to Grid object for manipulation
    try:
        current_grid = _parse_grid(output_list)
    except Exception:
        # Cannot even parse it into a Grid, return a minimal fallback
        return [[0]] 

    # --- Repair Sequence ---
    
    # 1. Crop-to-content: Fixes excessive padding/large size
    dsl = FinalHybridARCDSL() # Using a fresh DSL instance for safety
    repaired_grid = dsl.crop_to_content(current_grid)
    
    # 2. Re-check size constraints and validate
    repaired_list = repaired_grid.to_list()
    if _validate_grid_format(repaired_list):
        logger.info(f"AGV: Grid repaired via crop_to_content.")
        return repaired_list
        
    # 3. Size reduction (if still too large) - not generally applicable but safe fallback
    if repaired_grid.shape[0] > HybridARCConfig.MAX_GRID_SIZE or \
       repaired_grid.shape[1] > HybridARCConfig.MAX_GRID_SIZE:
       
        # As a last resort, resize to fit the max boundary (e.g., downscale by 2)
        # We skip this as resizing often destroys the solution logic. Return a minimal valid grid.
        return [[0]] 
        
    # If all repairs fail, return a fallback minimal grid
    logger.warning(f"AGV: Grid repair failed for task {task_id}.")
    return [[0]]


# --- 2. FINAL SUBMISSION GENERATOR ---

def generate_submission_file(tasks_dir: str = CHECKPOINT_DIR, 
                             submission_path: Path = SUBMISSION_FILE) -> bool:
    """
    Reads the final solved results, applies validation, and formats the final submission JSON.
    """
    
    # Load the checkpoint file which holds the latest solved_tasks dictionary
    checkpoint_file = Path(tasks_dir) / "arc_solver_checkpoint.json"
    
    if not checkpoint_file.exists():
        logger.error(f"Checkpoint file not found at {checkpoint_file}. Cannot generate submission.")
        return False
        
    try:
        with open(checkpoint_file, 'r') as f:
            checkpoint_data = json.load(f)
            solved_tasks = checkpoint_data.get("solved_tasks", {})
    except json.JSONDecodeError:
        logger.error("Error decoding checkpoint JSON.")
        return False

    final_submission_data = {}
    
    for task_id, output_list_of_lists in solved_tasks.items():
        # The solver stores a list of lists of outputs (one for each test input)
        processed_outputs = []
        for output_list in output_list_of_lists:
            # Apply Adaptive Grid Validation (AGV)
            processed_output = adaptive_grid_validation(task_id, output_list)
            processed_outputs.append(processed_output)
            
        final_submission_data[task_id] = processed_outputs

    # --- 3. MINIMAL SUBMISSION HEURISTIC (MSH) (NOVEL INSIGHT 9) ---

    def apply_msh(data: Dict[str, Any]) -> Dict[str, Any]:
        """
        NOVEL INSIGHT 9: Minimal Submission Heuristic (MSH).
        Ensures that only the first predicted output for each task is submitted, 
        as required by the ARC competition rules (only one attempt per test case).
        It also ensures that only *solved* tasks are submitted.
        """
        msh_data = {}
        for task_id, outputs in data.items():
            if outputs and len(outputs) > 0:
                # Only take the first predicted output grid
                msh_data[task_id] = [outputs[0]]
            # If outputs is empty (shouldn't happen if task was "solved"), we skip it
        return msh_data

    submission_ready_data = apply_msh(final_submission_data)

    # Write the final JSON file
    try:
        with open(submission_path, 'w') as f:
            json.dump(submission_ready_data, f)
        logger.critical(f"Final submission file successfully generated at {submission_path}.")
        logger.critical(f"Total tasks formatted for submission: {len(submission_ready_data)}")
        return True
    except Exception as e:
        logger.error(f"Failed to write submission file: {e}")
        return False

# --- FINAL EXECUTION BLOCK (Utility) ---
def finalize_submission_process():
    """Utility call to execute the final generator."""
    if generate_submission_file():
        logger.info("Submission generation process complete.")
    else:
        logger.error("Submission generation failed.")

# finalize_submission_process()
logger.info(f"Cell 9 executed: Implemented Final Submission Generator, Adaptive Grid Validation (AGV), and Minimal Submission Heuristic (MSH).")


2025-10-30 14:55:42,395 - INFO - Cell 9 executed: Implemented Final Submission Generator, Adaptive Grid Validation (AGV), and Minimal Submission Heuristic (MSH).


In [10]:
#10
# Assuming Cells 1-9 have been executed (All classes, functions, and global variables defined).
# Global Dependencies: HybridARCConfig, FinalHybridARCDSL, FinalNeuralGuidance,
# FinalBeamSearch, load_all_arc_tasks, run_solver, train_neural_guidance_system,
# finalize_submission_process.

# --- 1. UTILITY: CHECK FOR TRAINED MODEL ---

def _check_for_trained_model(path: Path) -> bool:
    """Checks if the neural guidance model has already been trained/saved."""
    return path.exists() and path.stat().st_size > 1024 # Simple size check


# --- 2. NOVEL INSIGHT 10: DYNAMIC ENVIRONMENT ADAPTATION (DEA) ---

def dynamic_environment_adaptation(config: 'HybridARCConfig', neural_guidance: 'FinalNeuralGuidance'):
    """
    NOVEL INSIGHT 10: Adjusts runtime parameters based on environment constraints
    (e.g., no GPU, no pre-trained model).
    """
    model_path = CHECKPOINT_DIR / "neural_guidance_model.pth"
    
    if not torch.cuda.is_available() and config.DEVICE != 'cpu':
        config.DEVICE = 'cpu'
        logger.warning("DEA: CUDA not found. Switching to CPU mode.")
        
    if config.DEVICE == 'cpu':
        # Reduce search intensity when on CPU
        config.BEAM_WIDTH = max(config.BEAM_WIDTH // 2, 4)
        config.MAX_TASK_TIME = min(config.MAX_TASK_TIME, 5.0)
        logger.warning(f"DEA: CPU mode activated. Reduced BEAM_WIDTH to {config.BEAM_WIDTH}.")
    
    if not _check_for_trained_model(model_path):
        # If no model is trained, force a quick training pass, or disable BSM entirely
        logger.warning("DEA: No pre-trained model found. Solver is highly handicapped.")
        if config.IS_TRAINING_ENABLED:
            logger.info("DEA: Enabling quick SSPS training to mitigate performance loss.")
            # Set a low number of epochs for fast pre-training
            TRAINING_CONFIG['EPOCHS'] = 3 
            TRAINING_CONFIG['DATA_SIZE'] = 5000
        else:
            logger.critical("DEA: BSM is disabled. Search relies purely on Symbolic Heuristics and PIS.")


# --- 3. NOVEL INSIGHT 11: TASK-LEVEL PROGRAM ABSTRACTION (TLPA) ---

def task_level_program_abstraction(program: Program, task: Task, dsl: FinalHybridARCDSL) -> Program:
    """
    NOVEL INSIGHT 11: Simplifies or confirms the final program by attempting to 
    reduce its complexity while maintaining correctness across all training pairs.
    (Placeholder: Full implementation is complex, we provide the core check)
    """
    if isinstance(program, FunctionalProgram) and len(program.steps) > 1:
        # Check if the final step is redundant (e.g., crop_to_content on an already-cropped grid)
        
        # TLPA Example: Redundant Final Steps
        simplified_steps = list(program.steps)
        
        # If the last two steps are identical, remove one (e.g., rotate_90, rotate_90 -> rotate_180)
        if len(simplified_steps) >= 2 and simplified_steps[-1].primitive == simplified_steps[-2].primitive:
            # Check for specific primitive redundancies (e.g. flip/rotate cancellations)
            if simplified_steps[-1].primitive == 'rotate_90' and len(program.steps) == 4:
                 # Check if the 4 x rotate_90 can be simplified to identity (too complex for final cell)
                 pass
            
        # The key check: ensure the simplified program is still 100% correct
        simplified_program = FunctionalProgram(simplified_steps)
        
        is_still_valid = True
        for input_grid, target_grid in task.train_pairs:
            output_simplified = simplified_program.execute(input_grid, dsl)
            # Use exact equality check for TLPA validation
            if output_simplified != target_grid: 
                is_still_valid = False
                break
                
        if is_still_valid:
            logger.info(f"TLPA: Program simplified from {len(program.steps)} to {len(simplified_steps)} steps.")
            return simplified_program
            
    return program # Return the original program if simplification fails or is not applicable


# --- 4. THE ULTIMATE EXECUTION FLOW ---

def full_solver_run(data_path: str = './data'):
    """
    The main orchestration function, executing the entire ARC solver pipeline.
    """
    logger.info("--- STARTING ARC PRIZE ULTIMATE SOLVER (V5.0 NEURO-SYMBOLIC EDITION) ---")
    start_time = time.time()

    # --- Step 1: Initialize and Adapt ---
    # Global Config (re-initialize in case of modifications)
    config = HybridARCConfig()
    
    # Initialize Core Components
    dsl = FinalHybridARCDSL()
    scorer = SemanticMetric()
    heuristic_scorer = SymbolicHeuristicScorer(dsl)
    neural_guidance = FinalNeuralGuidance()
    
    # Apply Dynamic Adaptation (Insight 10)
    dynamic_environment_adaptation(config, neural_guidance)
    
    # --- Step 2: Load Data and Pre-process ---
    all_tasks = load_all_arc_tasks(data_path)
    
    # Pre-extract features and apply HTG for ordering
    detector = FinalObjectDetector()
    for task in all_tasks.values():
        _pre_extract_features_and_cache(task, detector)
    sorted_task_ids = group_tasks_by_structure(all_tasks)
    ordered_tasks = {tid: all_tasks[tid] for tid in sorted_task_ids}

    # --- Step 3: Train Neural Guidance (if necessary) ---
    model_path = CHECKPOINT_DIR / "neural_guidance_model.pth"
    if not _check_for_trained_model(model_path) and config.IS_TRAINING_ENABLED:
        logger.info("Executing initial BSM pre-training...")
        train_neural_guidance_system(neural_guidance)
    elif _check_for_trained_model(model_path):
        # Load the pre-trained model state
        neural_guidance.load_state_dict(torch.load(model_path, map_location=config.DEVICE))
        neural_guidance.eval()
        logger.info("Loaded pre-trained BSM model.")


    # --- Step 4: Execute Main Search Loop ---
    solver = FinalBeamSearch(
        dsl=dsl, 
        scorer=scorer, 
        heuristic_scorer=heuristic_scorer, 
        neural_guidance=neural_guidance
    )
    
    # Run the main checkpointed loop
    run_solver(ordered_tasks, solver)
    
    # --- Step 5: Final Submission Generation ---
    finalize_submission_process()
    
    end_time = time.time()
    logger.critical(f"\n*** SOLVER RUN COMPLETE ***")
    logger.critical(f"Total Uptime: {(end_time - start_time):.1f} seconds.")
    logger.critical(f"Final results saved to {SUBMISSION_FILE.name}")


# === EXECUTION TRIGGER ===
# This is the single line that starts the entire pipeline.
if __name__ == '__main__':
    # To run this block in a real ARC environment, 
    # ensure the 'data' directory (with task JSONs) is accessible.
    
    # For this demonstration, we call the execution setup.
    # Note: Full execution requires defining the 'HybridARCConfig' 
    # class and all preceding functions/classes.
    try:
        full_solver_run()
    except NameError as e:
        print(f"\n*** EXECUTION ERROR: Missing prerequisite component from Cells 1-9. ***")
        print(f"Error details: {e}")
        print("Please ensure all prior cells (1-9) were executed successfully and their global definitions are available.")
    except Exception as e:
        print(f"\n*** FATAL RUNTIME ERROR ***")
        print(f"Details: {e}")

# logger.info(f"Cell 10 executed: Orchestrated the full ARC solver pipeline.")


2025-10-30 14:55:42,425 - INFO - --- STARTING ARC PRIZE ULTIMATE SOLVER (V5.0 NEURO-SYMBOLIC EDITION) ---
2025-10-30 14:55:42,462 - WARNING - DEA: CUDA not found. Switching to CPU mode.
2025-10-30 14:55:42,463 - WARNING - DEA: CPU mode activated. Reduced BEAM_WIDTH to 8.
2025-10-30 14:55:42,464 - WARNING - DEA: No pre-trained model found. Solver is highly handicapped.

*** FATAL RUNTIME ERROR ***
Details: 'HybridARCConfig' object has no attribute 'IS_TRAINING_ENABLED'
